In [13]:
import getpass
import os

if "LANGCHAIN_API_KEY" not in os.environ:
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [14]:
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass()

#### A. Datos de ejemplo

In [15]:
import os
import re
from pydantic import BaseModel, ValidationError
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain import hub

In [16]:
# Conectar a la base de datos de ejemplo
db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print("Dialect:", db.dialect)
print("Tablas disponibles:", db.get_usable_table_names())

Dialect: sqlite
Tablas disponibles: ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


In [17]:
# Modelo y plantilla de LangChain
llm = ChatOpenAI(model="gpt-4o-mini")
query_prompt_template = hub.pull("langchain-ai/sql-query-system-prompt")

# Modelo estructurado para la validación
class QueryOutput(BaseModel):
    query: str

In [18]:
# Limpieza de resultados en streaming
def clean_streamed_output(raw_output: str) -> str:
    """
    Limpia los decoradores de bloques de código (como ```sql) de un texto.
    """
    cleaned_output = re.sub(r"```[\w]*\n|```", "", raw_output).strip()
    return cleaned_output

# Función de streaming
def write_query_streaming(state: dict) -> str:
    """
    Genera un SQL query en streaming basado en el estado proporcionado.
    """
    # Crear el prompt inicial
    prompt = query_prompt_template.invoke({
        "dialect": db.dialect,
        "top_k": 10,
        "table_info": db.get_table_info(),
        "input": state["question"],
    })

    # Configuración del modelo en streaming
    streaming_llm = ChatOpenAI(model="gpt-4o-mini", streaming=True)

    # Acumular el resultado generado
    raw_output = ""
    print("Generando respuesta en streaming:")
    for token in streaming_llm.stream(prompt):
        raw_output += token.content
        print(token.content, end="")  # Mostrar token en tiempo real

    print("\n\nLimpieza de la salida...")
    cleaned_output = clean_streamed_output(raw_output)

    print("\nQuery limpio:")
    print(cleaned_output)
    return cleaned_output


In [19]:
# Función estructurada
def write_query_structured(state: dict) -> str:
    """
    Genera un SQL query usando with_structured_output para validación automática.
    """
    # Crear el prompt inicial
    prompt = query_prompt_template.invoke({
        "dialect": db.dialect,
        "top_k": 10,
        "table_info": db.get_table_info(),
        "input": state["question"],
    })

    # Configuración de LLM con esquema estructurado
    structured_llm = llm.with_structured_output(QueryOutput)

    # Invocación del modelo
    result = structured_llm.invoke(prompt)

    # Acceso al atributo query del objeto
    return result.query

In [20]:
# Orquestación combinada
def orchestrate_query(state: dict) -> str:
    """
    Combina streaming para feedback en tiempo real y structured output para validación.
    """
    # Paso 1: Generar query en streaming
    raw_query = write_query_streaming(state)

    # Paso 2: Validar query con structured output
    try:
        validated_query = write_query_structured({"question": raw_query})
        print("\nConsulta validada:", validated_query)
        return validated_query
    except ValidationError as e:
        print("\nError de validación:", e)
        raise ValueError("La consulta generada no es válida.")

In [23]:
question = "¿Cuántos empleados hay que su nombre empiece con w?"
try:
    result = orchestrate_query({"question": question})
    print("\nQuery final:", result)
except Exception as e:
    print("\nError:", e)    

Generando respuesta en streaming:
```sql
SELECT COUNT(*) AS EmployeeCount 
FROM Employee 
WHERE FirstName LIKE 'W%' 
LIMIT 10;
```

Limpieza de la salida...

Query limpio:
SELECT COUNT(*) AS EmployeeCount 
FROM Employee 
WHERE FirstName LIKE 'W%' 
LIMIT 10;

Consulta validada: SELECT COUNT(*) AS EmployeeCount 
FROM Employee 
WHERE FirstName LIKE 'W%' 
LIMIT 10;

Query final: SELECT COUNT(*) AS EmployeeCount 
FROM Employee 
WHERE FirstName LIKE 'W%' 
LIMIT 10;
